# User for Genre

- User for genre toma como parámetro un género particular, y retorna el usuario  con mayor tiempo jugado para dicho genero. Ademas de una lista de todos del tiempo acumulado de juego para un año de lanzamiento

- Para optimizar la api, crearemos una tabla auxiliar 
donde las filas son los generos diferentes, y tendrá como indice cada uno de los generos diferentes.

- **Observación:** No tenemos la información sobre el año en que un usuario jugó un juego, con la información que tenemos existen dos opciones. La primera es tomar solo los usuario que realizarón un review, en este caso podemos extraer el año en que realizo el post, y podemos "inferir" que el usuario jugo este año, aunque existen muchos jugadores de steam que nunca han realizado algun post (me incluyo).

- Por lo que vamos a tomar el año de lanzamiento del juego que a pesar de no tener tanto sentido podemos garantizar que para un juego que se lanzó en un año dado y que un usuario acumulo cierta cantidad de tiempo jugado para este año.



## Carga de data

In [3]:
import pandas as pd
import os

path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

In [28]:
## Cargamos la data necesaria para solucionar este problema.
steam_colums = ['id','genres','release_year']
steam_games = pd.read_csv(path_steam_games, usecols = steam_colums)

user_item_cols = ['item_id','user_id','playtime_forever']
user_items = pd.read_csv(path_user, usecols = user_item_cols)


## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'item_id'}, inplace=True)


In [29]:
user_items.isna().sum()

user_id             0
playtime_forever    0
item_id             0
dtype: int64

## Creación de tabla consulta

In [30]:
user_items

,user_id,playtime_forever,item_id
0,76561197970982479,0.10,10.0
1,76561197970982479,0.00,20.0
2,76561197970982479,0.12,30.0
3,76561197970982479,0.00,40.0
4,76561197970982479,0.00,50.0
...,...,...,...
5094077,76561198329548331,0.00,346330.0
5094078,76561198329548331,0.00,373330.0
5094079,76561198329548331,0.05,388490.0
5094080,76561198329548331,0.07,521570.0


In [31]:
steam_users = steam_games.merge(user_items,
                              on='item_id',
                              how='inner')

### Eliminación de usuarios ausentes
- Luego de agrupar la base de datos de los usuarios con la información de steam, vamos a eliminar los usuarios que nunca jugaros, esto ya que el objetivo de la consulta es encotrar el usuario que más tiempo jugó, por lo que estos usuarios ausentes no serán relevantes para nuestra consulta

In [32]:
steam_users = steam_users[steam_users['playtime_forever'] > 0 ]

- Para quedarme con los generos únicos debmos desempaquetar cada uno de los generos de de cada juego, luego crear una lista única con estos elementos.

In [33]:
genres_unique =  steam_games['genres'].str.split(', ').explode().unique()

In [34]:
steam_games['genres'].str.split(', ').explode().value_counts().head(10)

Indie               16783
Action              12320
Casual               9471
Adventure            9344
Simulation           7508
Strategy             7289
RPG                  5552
Singleplayer         4268
Multiplayer          2283
Great Soundtrack     2185
Name: genres, dtype: int64

In [35]:
len(genres_unique)

339

## Construcción de la consulta

- A continuación vamos a realizar las consultas necesarias para encontrar el usuario con mas horas jugadas por genero y la lista de tiempo acumulado por año de lanzamiento de juego.

- Para responder este problema vamos a ejemplificar la consulta inspecciónar con un genero particular, en este caso **Action**, luego vamos a construir la tabla para cada uno de los generos únicos.

In [42]:
genres = 'Action'

user_max = steam_users[steam_users['genres'].str.contains(genres)][['user_id','playtime_forever']].groupby('user_id').sum().nlargest(1,'playtime_forever').index[0]

In [43]:
user_max

'Sp3ctre'

- Dado el usuario encontrado anteriormente, filtraremos por este usuario y finalmente vamos a calcular la cantidad de horas jugadas por año para el usuario que jugo mas tiempo el genero dado.

In [44]:
play_to_years = steam_users[(steam_users['user_id'] == user_max)][['release_year','playtime_forever']].groupby('release_year').sum()

In [45]:
play_to_years.to_dict()

{'playtime_forever': {1995: 3.62,
  1997: 4.71,
  1999: 10.5,
  2000: 1177.4,
  2001: 4.17,
  2002: 5.890000000000001,
  2003: 129.06,
  2004: 2134.04,
  2005: 355.65,
  2006: 1536.3999999999999,
  2007: 1879.91,
  2008: 170.28,
  2009: 1818.46,
  2010: 1385.72,
  2011: 2591.25,
  2012: 6404.280000000001,
  2013: 3452.12,
  2014: 2285.46,
  2015: 8579.77,
  2016: 778.46,
  2017: 722.12}}

### Creacion de funciones
Lo realizado anteriormente fue para un género particular, vamos a realizar las funciones que realicen esto, luego vamos a aplciarlo con los 339 generos diferentes que tenemos

In [46]:

def user_max_time(genres):
  """ Retorna el usuario con mayor tiempo jugado para el genero dado.\n

  Args:
      genres (str): genero de juego.
  """
  try:
    user_max = steam_users[steam_users['genres'].str.contains(genres)][['user_id','playtime_forever']].groupby('user_id').sum().nlargest(1,'playtime_forever').index[0]
    print(genres)
    return(user_max)
  except IndexError:
    return('Nadie')


def max_time_for_year(user):
  if user != 'Nadie':
    return(steam_users[(steam_users['user_id'] == user)][['release_year','playtime_forever']].groupby('release_year').sum().to_dict())
  

### Creacion de tabla
- A continuación vamos a crear una tabla que contendra cada uno de los resultados, esta tabla será de gran utilidad ya que al desplegar la api, consumirá una memoria minima.

In [52]:
tabla_1 = pd.DataFrame(index=genres_unique,columns=['nombre'])

In [54]:
tabla_1 = tabla_1.reset_index()

In [55]:
tabla_1

,index,nombre
0,Strategy,NaN
1,Action,NaN
2,Indie,NaN
3,Casual,NaN
4,Simulation,NaN
...,...,...
334,Faith,NaN
335,Tutorial,NaN
336,Steam Machine,NaN
337,Bikes,NaN


In [56]:

tabla_1['nombre'] = tabla_1['index'].apply(user_max_time)


Strategy
Action
Indie
Casual
Simulation
Free to Play
RPG
Card Game
Trading Card Game
Turn-Based
Fantasy
Tactical
Dark Fantasy
Board Game
PvP
2D
Competitive
Replay Value
Character Customization
Female Protagonist
Difficult
Design & Illustration
Sports
Multiplayer
Adventure
FPS
Shooter
Third-Person Shooter
Sniper
Third Person
Pixel Graphics
Cute
Physics
Science
Early Access
VR
Racing
Classic
Gore
1990's
Singleplayer
Sci-fi
Aliens
First-Person
Story Rich
Atmospheric
Silent Protagonist
Great Soundtrack
Moddable
Linear
Retro
Funny
Clicker
Turn-Based Strategy
Gothic
Isometric
Stealth
Mystery
Assassin
Survival
Comedy
Stylized
City Builder
Building
Economy
Base Building
Education
Wargame
Cold War
Real-Time with Pause
RTS
Diplomacy
War
Psychological Horror
Sandbox
Co-op
Mod
Online Co-Op
Animation & Modeling
Massively Multiplayer
Puzzle
Horror
Management
Futuristic
Cyberpunk
Destruction
Music
Driving
Arcade
Mechs
Robots
Rome
Underground
Exploration
Point & Click
4X
Historical
Trains
Top-Down
Und

In [57]:
tabla_1

,index,nombre
0,Strategy,shinomegami
1,Action,Sp3ctre
2,Indie,REBAS_AS_F-T
3,Casual,REBAS_AS_F-T
4,Simulation,shinomegami
...,...,...
334,Faith,Nadie
335,Tutorial,Nadie
336,Steam Machine,REBAS_AS_F-T
337,Bikes,Nadie


In [213]:
tabla_1.to_csv('02_nombres__max.csv.gz')

In [58]:
tabla_1['year'] = tabla_1['nombre'].apply(max_time_for_year)

In [223]:
tabla_1[tabla_1['nombre']!='Nadie'].to_csv('02_endpoint_full.csv.gz')

In [59]:
tabla_1

,index,nombre,year
0,Strategy,shinomegami,"{'playtime_forever': {1988: 2.27, 1993: 513.71..."
1,Action,Sp3ctre,"{'playtime_forever': {1995: 3.62, 1997: 4.71, ..."
2,Indie,REBAS_AS_F-T,"{'playtime_forever': {1998: 0.02, 1999: 3.75, ..."
3,Casual,REBAS_AS_F-T,"{'playtime_forever': {1998: 0.02, 1999: 3.75, ..."
4,Simulation,shinomegami,"{'playtime_forever': {1988: 2.27, 1993: 513.71..."
...,...,...,...
334,Faith,Nadie,None
335,Tutorial,Nadie,None
336,Steam Machine,REBAS_AS_F-T,"{'playtime_forever': {1998: 0.02, 1999: 3.75, ..."
337,Bikes,Nadie,None


In [4]:
import ast
def user_for_genres(genres):
  path_endpoint_02 = os.path.join('..','data','clear','02_user_for_genre_data.csv.gz')
  
  consulta_02 = pd.read_csv(path_endpoint_02,index_col=['index'])
    
  user_max = consulta_02.loc['Action'].nombre
  
  diccionario = ast.literal_eval(consulta_02.loc['Action'].year)

  diccionario['Horas_Jugadas']  = diccionario.pop('playtime_forever')

  return ({f"Usuario con más horas jugadas para Género {genres}" : user_max ,"Horas Jugadas" : diccionario['Horas_Jugadas']}) 

In [5]:
user_for_genres('Action')

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre',
 'Horas Jugadas': {1995: 3.62,
  1997: 4.71,
  1999: 10.5,
  2000: 1177.4,
  2001: 4.17,
  2002: 5.890000000000001,
  2003: 129.06,
  2004: 2134.04,
  2005: 355.65,
  2006: 1536.3999999999999,
  2007: 1879.91,
  2008: 170.28,
  2009: 1818.46,
  2010: 1385.72,
  2011: 2591.25,
  2012: 6404.280000000001,
  2013: 3452.12,
  2014: 2285.46,
  2015: 8579.77,
  2016: 778.46,
  2017: 722.12}}